In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
from sklearn.cross_validation import KFold
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
import xgboost as xgb
import sys
import kaggle_xgb
import calc_ndcg
from sklearn.grid_search import ParameterGrid

Read in just the testing and training data.

In [34]:
#train_subset = pd.read_csv('../data/train_users_2.csv', nrows = 100000, index_col='id')
train_data = pd.read_csv('../data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('../data/test_users.csv', index_col = 'id')
sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('../data/age_gender_bkts.csv')
#countries = pd.read_csv('../data/countries.csv')

all_df is a pickled obect where the training and test has already been concatenated onall columns except 'country_destination'

In [53]:
sessions.dropna(subset=['user_id'], inplace = True)

sessions_tr = sessions[sessions.user_id.isin(train_data.index)]
sessions_test = sessions[sessions.user_id.isin(test_data.index)]
a_ignore = set(sessions_tr.action.unique()) ^ set(sessions_test.action.unique())
#sessions.groupby('action')['action_detail']

#set(sessions_test.action_detail.unique()) ^ set(sessions_tr.action_detail.unique())
#sessions_test.action_detail.unique()
#sessions_tr.action_detail.unique()
#train_data['date_account_created'] = pd.to_datetime(train_data.date_account_created)
#x = train_data[train_data.date_account_created>='2014']
#train_data[train_data.index.isin(sessions_tr.user_id)].timestamp_first_active

In [78]:
#s = sessions.groupby('action')['action_type'].nunique()
s[s>1]
sessions_tr[sessions_tr.action == 'approve'].user_id

284848     6udv3scuxe
284944     6udv3scuxe
3657667    yxf0sm9sbw
3657670    yxf0sm9sbw
3657671    yxf0sm9sbw
Name: user_id, dtype: object

In [97]:
actions50 = [sessions_test.groupby('action')['user_id'].nunique() >50]
print 'Number of unique actions with more than 50 users:', np.sum(sessions_test.groupby('action')['user_id'].nunique()>50)

Number of unique actions with more than 50: 206


In [102]:
#Let's get the actions with just one action type and one action detail.
print 'Number of unique actions with just one action type:'
np.sum(sessions_test.groupby('action')['action_detail'].nunique()==1)

Number of unique actions with just one action type:


273

In [103]:
sessions_test.groupby('action')['action_detail'].nunique()==1

action
10                                    True
11                                    True
12                                    True
15                                    True
about_us                              True
accept_decline                        True
account                               True
acculynk_bin_check_failed             True
acculynk_bin_check_success            True
acculynk_load_pin_pad                 True
acculynk_pin_pad_error                True
acculynk_pin_pad_inactive             True
acculynk_pin_pad_success              True
acculynk_session_obtained             True
active                                True
add_business_address_colorbox         True
add_guest_colorbox                    True
add_guests                            True
add_note                              True
agree_terms_check                     True
agree_terms_uncheck                   True
airbnb_picks                          True
ajax_check_dates                      True
ajax

In [114]:
train_data.loc[train_data.index.isin(sessions_tr[sessions_tr.action == 'new'].user_id), 'country_destination']

id
unkvmpwxo2      NDF
2pw9yjm8aq    other
ckkcg778ej      NDF
3n3wz1i5y6       US
qdzx8bye52      NDF
kv5nk5t1bq      NDF
hgao67zp6q      NDF
dywez4oem4      NDF
6k8ov8dr9k      NDF
98jeibnoq6       FR
w7kxmjecn9      NDF
6rg5dn6y92      NDF
kto9ybgvnv       US
0mj2n4z537       US
z7te7g1air      NDF
56il3dlb84       US
8l7zky3942       US
5kj9qbd85v    other
l3y5x8vugi      NDF
zlv8f1qg2g      NDF
woy4x73nen      NDF
4db6594ely      NDF
x04fkf40pw      NDF
jzkbk40w1q      NDF
cmjnbto6zg      NDF
mrsbt5pb9l       US
sjecbinecs       US
aynf03zznc      NDF
k3uht68h7u       US
t55nxx8n0s      NDF
              ...  
g7ryqvkeii      NDF
m4tbin4qln      NDF
qsvhjlat8h      NDF
icrzw93ddc      NDF
84wh2czudf    other
bhcu9fsa5j       US
xsx2kqtmvu      NDF
y8a5sozavm      NDF
hj3ko9isnt       US
iwpwc2wllf      NDF
5lu4h1ksk7      NDF
ldvksg216r      NDF
ixvti9ek0f      NDF
sasvxitm97      NDF
3talm5hohz      NDF
e0110dx6c9      NDF
s4wtj1tq5y      NDF
3zi316u9ag      NDF
2g5fa9ykpi      N

In [104]:
#find the number of users in test data with an action that is not found in the training sessions data
#We should look to find similar items to substitute them
for i in a_ignore:
    users = sessions_test[sessions_test.action == i].user_id.unique()
    if len(users) > 0:
        print i, len(users)

stpcv 1
acculynk_bin_check_failed 1
rest-of-world 1
report 1
view 1436
acculynk_pin_pad_success 1
accept_decline 1
my_reservations 203
this_hosting_reviews_3000 727
add_business_address_colorbox 9
events 1
host_cancel 1
braintree_client_token 89
deactivated 1
set_minimum_payout_amount 1
refund_guest_cancellation 1
revert_to_admin 1
acculynk_pin_pad_error 4
support_phone_numbers 8
custom_recommended_destinations 9667
sandy 1
reset_calendar 2
hard_fallback_submit 3
unsubscribe 1
business_travel 12
add_guest_colorbox 5
book 82
south-america 1


In [62]:
actions = list[set(sessions.action.unique())-a_ignore]
actions.remove(np.nan)
for a in actions:
    print sessions[sessions.action == a].user_id.groupby()
    break

TypeError: 'type' object has no attribute '__getitem__'

In [135]:
#iterate through each action and find those that have more than one action detail/ action type
#s = sessions.groupby('action')['action_type'].nunique()
#sessions.groupby('action')['action_detail'].nunique()

In [77]:
np.sum(sessions.loc[0:127, 'secs_elapsed'])

3430268.0

In [14]:
train_data[train_data.index.isin(sessions[sessions.action== 'personalize'].user_id)].country_destination.value_counts(normalize = True)

NDF      0.560603
US       0.297091
other    0.058018
FR       0.024556
IT       0.016707
GB       0.012078
ES       0.011937
CA       0.007073
DE       0.004347
NL       0.003924
AU       0.002303
PT       0.001363
Name: country_destination, dtype: float64

In [4]:
#convert targets to numerical values
targets = train_subset['country_destination']
targets_le = preprocessing.LabelEncoder()
targets = targets_le.fit_transform(targets)

In [5]:
train_X, train_Y = train_encoded.values, targets
xg_train = xgb.DMatrix(train_X, label = train_Y, missing = -1)

In [ ]:
#kfold cross validation
kf = KFold(len(train_subset), n_folds=10, shuffle=True, random_state=None)
param = {'num_class': 12, 'silent': 1, 'objective': 'multi:softprob'}
param['subsample'] = .5
param['col_sample_bytree'] = .5
param['eta'] = None
param['max_delta_step'] = None
param['max_depth'] = None

#grid search through the following parameter values
eta = [.15, .2, .25, .3, .35, .4, .5]
num_rounds = range(10,51,5)
max_depth = [5, 6, 7]
max_delta_step = [0, .5, 1.5, 3, 5]

#create data frame for results
col_names = ['eta', 'rounds','depth', 'delta']
col_names += range(10)
res_df = pd.DataFrame(columns=col_names)

i = 0
for delta in max_delta_step:
    param['max_delta_step'] = delta
    for depth in max_depth:
        print depth
        param['max_depth'] = depth
        for n in num_rounds:
            for e in eta:
                param['eta'] = e
                cv_error = []
                for train_index, test_index in kf:
                    #split data
                    train_X, train_Y = all_df.iloc[train_index,:], targets[train_index]
                    test_X, test_Y = all_df.iloc[test_index,:], targets[test_index]
                    xg_train = xgb.DMatrix(train_X, label = train_Y, missing = -1)
                    xg_test = xgb.DMatrix(test_X, missing = -1)
                    #run xgboost
                    bst = xgb.train(param, xg_train, num_boost_round = n)
                    #evaluate cross validation error on this split
                    pred_prob = bst.predict(xg_test)
                    cv_error.append(eval_error(pred_prob, test_Y))
                res_df.loc[i] = [e] + [n] + [depth] + [delta] + cv_error
                i += 1
                res_df.to_pickle('cross_valresults')

In [87]:
np.unique(targets)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [171]:
#pd.DataFrame(columns=['row_above', 'row_below', ])
results.sort_values(by= 'test-error-mean')

,test-error-mean,test-error-std,train-error-mean,train-error-std
5,0.211443,0.036614,0.156609,0.003926
9,0.212702,0.040006,0.148377,0.002375
8,0.212941,0.037422,0.150010,0.003367
7,0.213027,0.038918,0.152337,0.003187
6,0.214822,0.039571,0.153975,0.002795
3,0.214927,0.034995,0.164401,0.003206
4,0.216651,0.039865,0.159727,0.004517
2,0.218139,0.033415,0.169041,0.004995
1,0.220312,0.028580,0.177505,0.006153
0,0.232952,0.028746,0.194168,0.007831


In [135]:
def ndcg(preds, labels):
    """Calculate sum of normalzied discounted cumulative gain for the predictions
    The correct prediction will have a relevance of 1, incorrect predictions will have a relevance of 0.
    Weight the relevance values such that it is reduced logarithmically proportional to it its position     
    
    Args:
        preds: n*5 array of predictor targets
        labels: n*1 array targets
    Returns:
        sum of normalized discounted cumulative gain for all the predictions
    """
    
    #find positions where the prediction matches the label
    relv_pos = np.where(np.equal(preds, labels))[1]
    #weight func: log2(i+1), add one more to adjust for 0-based indexing
    total_ndcg = np.sum(1./np.log2(relv_pos+2))
    return total_ndcg
    
def evalerror(cls_prob, dtrain):
    """find top k predictions from probability matrix and call ndcg to find accuracy of predictions
    
    Args:
        cls_prob: 2D array, probability of each class for each person (n persons by m classes),
                  the column index corresponds with class
        labels: labels for the n persons
    returns:
        prediction accuracy using ndcg to evaluate predictions of each AirBNB user
    """
    #determine the top k predictions
    labels = dtrain.get_label()
    k = 5
    top_k = cls_prob.argsort(axis = 1)[:,:k:-1]
    #convert true values  and compared with predictions to check for equality
    labels = labels[:, None]
    return 'error', 1.-ndcg(top_k, labels)/len(labels)

In [ ]:
def non_NDF():
    '''remove users with destination NDF'''

    #remove users with destination NDF
    train_df = all_df.iloc[train_df['country_destination'] != 'NDF', :]
    eta = np.linspace(.15, .30, 10)
    param_grid = {}
    param_grid['eta'] = eta
    param_grid['max_depth'] = [5, 6, 7, 8]
    param_grid['subsample'] = [.5, .6, .7, .8]
    param_grid['colsample_bytree'] = [.5, .6, .7, .8]

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]